In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('./gdrive/MyDrive/AEGAN/AE_GAN_IAENG')
!nvidia-smi
!ls

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import DeepFool
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10
from art.defences.trainer import AdversarialTrainer
from art.defences.trainer import AdversarialTrainerMadryPGD

In [ ]:
class CifarCNN(nn.Module):

    def __init__(self):
        super(CifarCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.fc5 = nn.Linear(512, 256)
        self.fc6 = nn.Linear(256, 256)
        self.fc7 = nn.Linear(256, 10)

    def forward(self, x):
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.relu(self.bn2(self.conv2(h)))
        h = F.max_pool2d(h, 4)

        h = F.relu(self.bn3(self.conv3(h)))
        h = F.relu(self.bn4(self.conv4(h)))
        h = F.max_pool2d(h, 4)

        h = F.relu(self.fc5(h.view(h.size(0), -1)))
        h = F.relu(self.fc6(h))
        h = self.fc7(h)
        return F.log_softmax(h,dim=1)

In [ ]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_cifar10()
x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

In [ ]:
model = CifarCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)

In [ ]:
classifier.fit(x_train, y_train, batch_size=128, nb_epochs=50)

In [ ]:
predictions1 = classifier.predict(x_test)
accuracy1 = np.sum(np.argmax(predictions1, axis=1) == np.argmax(y_test, axis=1))/len(y_test)
print('Accuracy1 test examples:{}%'.format(accuracy1*100))

Accuracy1 test examples:79.2%


In [ ]:
attack_fgsm1 = FastGradientMethod(estimator=classifier,eps=0.1)
attack_fgsm2 = FastGradientMethod(estimator=classifier,eps=0.3)
attack_fgsm3 = FastGradientMethod(estimator=classifier,eps=0.5)
attack_fgsm4 = FastGradientMethod(estimator=classifier,eps=0.7)
attack_fgsm5 = FastGradientMethod(estimator=classifier,eps=0.01)
attack_fgsm6 = FastGradientMethod(estimator=classifier,eps=0.03)
attack_fgsm7 = FastGradientMethod(estimator=classifier,eps=0.05)
attack_fgsm8 = FastGradientMethod(estimator=classifier,eps=0.07)
fgsm_at = AdversarialTrainer(classifier, [attack_fgsm1,attack_fgsm2,attack_fgsm3,attack_fgsm4,attack_fgsm5,attack_fgsm6,attack_fgsm7,attack_fgsm8], ratio=0.5)
fgsm_at.fit(x_train, y_train, batch_size=128, nb_epochs=50)

# pgd_at = AdversarialTrainerMadryPGD(classifier, nb_epochs=300, batch_size=128, eps = 8, eps_step = 2, max_iter = 7)
# pgd_at.fit(x_train, y_train)

In [ ]:
predictions2 = classifier.predict(x_test)
accuracy2 = np.sum(np.argmax(predictions2, axis=1) == np.argmax(y_test, axis=1))/len(y_test)
print('Accuracy2 on test examples:{}%'.format(accuracy2*100))

In [ ]:
# attack = FastGradientMethod(estimator=classifier,eps=0.01)
# attack = BasicIterativeMethod(estimator=classifier, eps=0.01,eps_step=0.01)
# attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma=0.7)
# attack = CarliniL2Method(classifier=classifier, max_iter=500)
attack = DeepFool(classifier=classifier)                                                
adv = attack.generate(x=x_test[0:2000])
# cw = torch.load('./CW2.tar')
# adv = cw['CW']
# y_test = cw['y']
predictions3 = classifier.predict(adv)
accuracy3 = np.sum(np.argmax(predictions3, axis=1) == np.argmax(y_test[0:2000], axis=1))/len(y_test[0:2000])
# accuracy3 = np.sum(np.argmax(predictions3, axis=1) == y_test.numpy())/len(y_test)
print('Accuracy3 on test examples:{}%'.format(accuracy3*100))

Accuracy3 on test examples:37.0%
